# Hospital Length of Stay

In order for hospitals to optimize resource allocation, it is important to predict accurately how long a newly admitted patient will stay in the hospital.

This notebook takes advantage of the power of SQL Server and RevoScaleR (Microsoft R Server). The tables are all stored in a SQL Server, and most of the computations are done by loading chunks of data in-memory instead of the whole dataset.

It does the following: 

 * **Step 0: Packages and Compute Contexts**
 * **Step 1: Processing and Cleaning**
 * **Step 2: Feature Engineering**
 * **Step 3: Training and Evalutating the Models**

## Step 0: Packages and Compute Contexts

#### In this step, we set up the connection string to access a SQL Server Database and load the necessary library. 

In [133]:
# WARNING.
# We recommend not using Internet Explorer as it does not support plotting, and may crash your session.

In [174]:
# Load package.
library(RevoScaleR)

In [175]:
# Choose a database name and create it. 
db <- "Hospital"

## Connect to the master database only to create a new database. Change UID and PWD if you modified them. 
connection_string <- "Driver=SQL Server;Server=localhost;Database=master;UID=rdemo;PWD=D@tascience"

## Open a connection with SQL Server to be able to write queries with the rxExecuteSQLDDL function.
outOdbcDS <- RxOdbcData(table = "NewData", connectionString = connection_string, useFastRead=TRUE)
rxOpen(outOdbcDS, "w")

query <- sprintf( "if not exists(SELECT * FROM sys.databases WHERE name = '%s') CREATE DATABASE %s;", db, db)

## Create database. 
rxExecuteSQLDDL(outOdbcDS, sSQLString = query)

[1] TRUE

[1] TRUE

In [176]:
# Define Compute Contexts: user to input Server Name, database name, UOD and Password. 
connection_string <- sprintf("Driver=SQL Server;Server=localhost;Database=%s;UID=rdemo;PWD=D@tascience", db)
sql <- RxInSqlServer(connectionString = connection_string)
local <- RxLocalSeq()

## Open a connection with SQL Server to be able to write queries with the rxExecuteSQLDDL function in the new database.
outOdbcDS <- RxOdbcData(table = "NewData", connectionString = connection_string, useFastRead=TRUE)
rxOpen(outOdbcDS, "w")

[1] TRUE

#### The function below can be used to get the top n rows of a table stored on SQL Server. 
#### You can execute this cell throughout your progress by removing the comment "#", and inputting:
#### - the table name.
#### - the number of rows you want to display.

In [143]:
 display_head <- function(table_name, n_rows){
   table_sql <- RxSqlServerData(sqlQuery = sprintf("SELECT TOP(%s) * FROM %s", n_rows, table_name), connectionString = connection_string)
   table <- rxImport(table_sql)
   print(table)
}

# table_name <- "insert_table_name"
# n_rows <- 10
# display_head(table_name, n_rows)

## Step 1: Pre-Processing and Cleaning

In this step, we: 

**1.** Upload the data set to SQL.

**2.** Clean the merged data set: we replace NAs with the mode (categorical variables) or mean (continuous variables).

**Input:**  Data Set LengthOfStay.csv

**Output:** Cleaned raw data set LoS.

In [144]:
# Set the compute context to Local. 
rxSetComputeContext(local)

In [140]:
# Upload the data set to SQL.

## Specify the desired column types. 
## Character and Factor are converted to nvarchar(255), Integer to Integer and Numeric to Float. 
column_types <-  c(eid = "integer",               
                   vdate = "character",           
                   rcount = "integer",        
                   gender = "factor",            
                   dialysisrenalendstage = "factor",             
                   asthma = "factor",                
                   irondef = "factor",                   
                   pneum = "factor",                 
                   substancedependence = "factor",                  
                   psychologicaldisordermajor = "factor",             
                   depress = "factor",           
                   psychother = "factor",        
                   fibrosisandother = "factor",          
                   malnutrition = "factor",                               
                   hemo = "numeric",            
                   hematocritic = "numeric",           
                   neutrophils = "numeric",           
                   sodium = "numeric",          
                   glucose = "numeric",             
                   bloodureanitro = "numeric",                 
                   creatinine = "numeric",                 
                   bmi = "numeric",                 
                   pulse = "numeric",                  
                   respiration = "numeric",                  
                   secondarydiagnosisnonicd9 = "factor",                   
                   lengthofstay = "factor")


## Point to the input data set while specifying the classes.
LoS_text <- RxTextData(file = "LengthOfStay.csv", colClasses = column_types)

## Upload the table to SQL. 
LengthOfStay_sql <- RxSqlServerData(table = "LengthOfStay", connectionString = connection_string)
rxDataStep(inData = LoS_text, outFile = LengthOfStay_sql, overwrite = TRUE)

print("Data exported to SQL")

Rows Read: 100000, Total Rows Processed: 100000, Total Chunk Time: 2.874 seconds 

Elapsed time to compute low/high values and/or factor levels: 3.171 secs.
 
Total Rows written: 100000, Total time: 9.414
Rows Read: 100000, Total Rows Processed: 100000, Total Chunk Time: 11.803 seconds 
[1] "Data exported to SQL"


In [150]:
# Clean the Merged data set: replace NAs with the mode (categorical variables) or mean (continuous variables). 

fill_NA_mode_mean <- function(table = "LengthOfStay"){
  
  ## Get the variables names and types. Assumption: no NA in eid. 
  data_sql <- RxSqlServerData(table = table, connectionString = connection_string)
  col <- rxCreateColInfo(data_sql)
  colnames <- names(col)
  var <- colnames[!colnames %in% c("eid")]
  
  categ_names <- c()
  contin_names <- c()
  for(name in var){
    if(col[[name]]$type == "numeric"){
      contin_names[length(contin_names) + 1] <- name
    } else{
      categ_names[length(categ_names) + 1] <- name
    }
  }
   
  ## For Categoricals: 
  ### Compute the modes and insert them in a table called Modes.
  rxExecuteSQLDDL(outOdbcDS, sSQLString = paste("DROP TABLE if exists Modes;"
                                                , sep=""))
  
  rxExecuteSQLDDL(outOdbcDS, sSQLString = paste("CREATE TABLE Modes
                                                (name varchar(30),
                                                 mode varchar(30));"
                                                , sep=""))
  
  for(name in categ_names){
    rxExecuteSQLDDL(outOdbcDS, sSQLString = sprintf("INSERT INTO Modes
                                                     SELECT '%s', mode
                                                     FROM (SELECT TOP(1) %s as mode, count(*) as cnt
                                                           FROM %s
                                                           GROUP BY %s 
                                                           ORDER BY cnt desc) as t;",name, name, table, name))
  }
 ### Import the Modes table.   
  Modes_sql <- RxSqlServerData(table = "Modes", connectionString = connection_string) 
  Modes <- rxImport(Modes_sql)
    
 ### Replace the NULL with the modes. 
  for(name in categ_names){
    mode <- Modes[Modes$name == name ,2]
    rxExecuteSQLDDL(outOdbcDS, sSQLString = sprintf("UPDATE %s
                                                     SET %s = ISNULL(%s, '%s' )
                                                      ;",table, name, name, mode))
  }
  
  ## For Continuous: 
  ### Compute the means and insert them in a table called Means.
  rxExecuteSQLDDL(outOdbcDS, sSQLString = paste("DROP TABLE if exists Means;"
                                                , sep=""))
  
  rxExecuteSQLDDL(outOdbcDS, sSQLString = paste("CREATE TABLE Means
                                                (name varchar(30),
                                                mean float);"
                                                , sep=""))
  
  for(name in contin_names){
    rxExecuteSQLDDL(outOdbcDS, sSQLString = sprintf("INSERT INTO Means
                                                    SELECT '%s', mean
                                                    FROM (SELECT AVG(%s) as mean
                                                    FROM %s) as t;",name, name, table))
  }
  ### Import the Means table.   
  Means_sql <- RxSqlServerData(table = "Means", connectionString = connection_string) 
  Means <- rxImport(Means_sql)
  
  ### Replace the NULL with the means. 
  for(name in contin_names){
    mean <- Means[Means$name == name ,2]
    rxExecuteSQLDDL(outOdbcDS, sSQLString = sprintf("UPDATE %s
                                                    SET %s = ISNULL(%s, %s )
                                                    ;",table, name, name, mean))
  }

}
  
# Apply the function to LengthOfStay. 
fill_NA_mode_mean()  

print("Data cleaned")

Rows Read: 17, Total Rows Processed: 17, Total Chunk Time: Less than .001 seconds 
Rows Read: 10, Total Rows Processed: 10, Total Chunk Time: Less than .001 seconds 
[1] "Data cleaned"


## Step 2: Feature Engineering

In this step, we:

**1.** Standardize the continuous variables (Z-score).

**2.** Create the variable number_of_issues: the number of preidentified medical conditions.
 

**Input:** Data set before feature engineering LengthOfStay.

**Output:** Data set with new features LoS.

In [151]:
# Create new features. 
# We use this opportunity to convert nvarchar(255) variables to char(1) for more efficient storage. 

rxExecuteSQLDDL(outOdbcDS, sSQLString = paste("DROP TABLE if exists LoS;"
, sep=""))

rxExecuteSQLDDL(outOdbcDS, sSQLString = paste(
        "SELECT eid, vdate, rcount, CAST(gender as char(1)) AS gender,
         CAST(dialysisrenalendstage as char(1)) AS dialysisrenalendstage, CAST(asthma as char(1)) AS asthma, 
         CAST(irondef as char(1)) AS irondef, CAST(pneum as char(1)) AS pneum, 
         CAST(substancedependence as char(1)) AS substancedependence,
         CAST(psychologicaldisordermajor as char(1)) AS psychologicaldisordermajor,
         CAST(depress as char(1)) AS depress, CAST(psychother as char(1)) AS psychother,
         CAST(fibrosisandother as char(1)) AS fibrosisandother, CAST(malnutrition as char(1)) AS malnutrition,
         (hemo - AVG(hemo) OVER())/(STDEV(hemo) OVER()) AS hemo_s,
         (hematocritic - AVG(hematocritic) OVER())/(STDEV(hematocritic) OVER()) AS hematocritic_s,
         (neutrophils - AVG(neutrophils) OVER())/(STDEV(neutrophils) OVER()) AS neutrophils_s,
         (sodium - AVG(sodium) OVER())/(STDEV(sodium) OVER()) AS sodium_s,
         (glucose - AVG(glucose) OVER())/(STDEV(glucose) OVER()) AS glucose_s,
         (bloodureanitro - AVG(bloodureanitro) OVER())/(STDEV(bloodureanitro) OVER()) AS bloodureanitro_s,
         (creatinine - AVG(creatinine) OVER())/(STDEV(creatinine) OVER()) AS creatinine_s,
         (bmi - AVG(bmi) OVER())/(STDEV(bmi) OVER()) AS bmi_s,
         (pulse - AVG(pulse) OVER())/(STDEV(pulse) OVER()) AS pulse_s,
         (respiration - AVG(respiration) OVER())/(STDEV(respiration) OVER()) AS respiration_s,
         CAST((CAST(dialysisrenalendstage as int) + CAST(asthma as int) + CAST(irondef as int) + CAST(pneum as int) +
          CAST(substancedependence as int) + CAST(psychologicaldisordermajor as int) + CAST(depress as int) +
          CAST(psychother as int) + CAST(fibrosisandother as int) + CAST(malnutrition as int)) as varchar(2)) 
         AS number_of_issues,
         CAST(secondarydiagnosisnonicd9 as varchar(2)) AS secondarydiagnosisnonicd9, discharged, facid,
         CAST(lengthofstay as char(1)) AS lengthofstay
INTO LoS 
FROM LengthOfStay;"
, sep=""))


print("Feature Engineering Completed")

[1] TRUE

[1] TRUE

[1] "Feature Engineering Completed"


## Step 3-A: Training and Evaluating the Models: Classification Approach

In this step we:

**1.** Split LoS into a Training LoS_Train, and a Testing set LoS_Test.  

**2.** Train classification Random Forest (RF) and Gradient Boosting Trees (GBT) on LoS_Train, and save them to SQL. 

**3.** Score RF and GBT on LoS_Test.

**Input:** Data set LoS

**Output:** Random forest and GBT models saved to SQL. 

In [177]:
# Point to the SQL table with the data set for modeling. Strings will be converted to factors.
LoS <- RxSqlServerData(table = "LoS", connectionString = connection_string, stringsAsFactors = T)

# The target variable should be a factor for classification.
rxExecuteSQLDDL(outOdbcDS, sSQLString = paste("ALTER TABLE LoS ALTER COLUMN lengthofstay char(1);", sep=""))

# Get variable names, types, and levels for factors.
column_info <- rxCreateColInfo(LoS)

print("Column information received")

[1] TRUE

[1] "Column information received"


In [178]:
# Randomly split the data into a training set and a testing set, with a splitting % p.
# p % goes to the training set, and the rest goes to the testing set. Default is 70%. 

p <- "70" 

## Open a connection with SQL Server to be able to write queries with the rxExecuteSQLDDL function.
outOdbcDS <- RxOdbcData(table = "NewData", connectionString = connection_string, useFastRead=TRUE)
rxOpen(outOdbcDS, "w")

## Create the Train_Id table containing Lead_Id of training set. 
rxExecuteSQLDDL(outOdbcDS, sSQLString = paste("DROP TABLE if exists Train_Id;", sep=""))

rxExecuteSQLDDL(outOdbcDS, sSQLString = sprintf(
  "SELECT eid
   INTO Train_Id
   FROM LoS
   WHERE ABS(CAST(BINARY_CHECKSUM(eid, NEWID()) as int)) %s < %s ;"
  ,"% 100", p ))

## Point to the training set. It will be created on the fly when training models. 
LoS_Train <- RxSqlServerData(  
  sqlQuery = "SELECT *   
              FROM LoS 
              WHERE eid IN (SELECT eid from Train_Id)",
  connectionString = connection_string, colInfo = column_info)

## Point to the testing set. It will be created on the fly when testing models. 
LoS_Test <- RxSqlServerData(  
  sqlQuery = "SELECT *   
              FROM LoS 
              WHERE eid NOT IN (SELECT eid from Train_Id)",
  connectionString = connection_string, colInfo = column_info)

print("Splitting completed")

[1] TRUE

[1] TRUE

[1] TRUE

[1] "Splitting completed"


In [179]:
# Specify the variables to keep for the training by writing the formula.

variables_all <- rxGetVarNames(LoS)
variables_to_remove <- c("eid", "vdate", "discharged")
traning_variables <- variables_all[!(variables_all %in% c("lengthofstay", variables_to_remove))]
formula <- as.formula(paste("lengthofstay ~", paste(traning_variables, collapse = "+")))

In [180]:
# Compute Context is set to SQL for model training.
rxSetComputeContext(sql)

In [181]:
# Train a Random Forest.
forest_model_class <- rxDForest(formula = formula,
                                data = LoS_Train,
                                nTree = 40,
                                minSplit = 10,
                                minBucket = 5,
                                cp = 0.00005,
                                seed = 5)


print("Training Classification RF done")

[1] "Training Classification RF done"


In [182]:
# Save the Random Forest in SQL. The compute context is set to Local in order to export the model. 
rxSetComputeContext(local)
saveRDS(forest_model_class, file = "forest_model_class.rds")
forest_model_class_raw <- readBin("forest_model_class.rds", "raw", n = file.size("forest_model_class.rds"))
forest_model_class_char <- as.character(forest_model_class_raw)
forest_model_class_sql <- RxSqlServerData(table = "Forest_Model_Class", connectionString = connection_string) 
rxDataStep(inData = data.frame(x = forest_model_class_char ), outFile = forest_model_class_sql, overwrite = TRUE)

# Set back the compute context to SQL.
rxSetComputeContext(sql)

print("Classification RF model uploaded to SQL")

Rows Read: 1744772, Total Rows Processed: 1744772
Total Rows written: 100000, Total time: 2.264
Total Rows written: 200000, Total time: 4.482
Total Rows written: 300000, Total time: 7.684
Total Rows written: 400000, Total time: 9.902
Total Rows written: 500000, Total time: 12.119
Total Rows written: 600000, Total time: 14.462
Total Rows written: 700000, Total time: 16.774
Total Rows written: 800000, Total time: 19.023
Total Rows written: 900000, Total time: 21.335
Total Rows written: 1000000, Total time: 23.63
Total Rows written: 1100000, Total time: 25.879
Total Rows written: 1200000, Total time: 28.144
Total Rows written: 1300000, Total time: 30.502
Total Rows written: 1400000, Total time: 32.751
Total Rows written: 1500000, Total time: 34.969
Total Rows written: 1600000, Total time: 37.218
Total Rows written: 1700000, Total time: 39.435
Total Rows written: 1744772, Total time: 40.451
, Total Chunk Time: 40.482 seconds 
[1] "Classification RF model uploaded to SQL"


In [183]:
# Train a Gradient Boosted Trees model.
btree_model_class <- rxBTrees(formula = formula,
                              data = LoS_Train,
                              learningRate = 0.05,
                              minSplit = 10,
                              minBucket = 5,
                              cp = 0.0005,
                              nTree = 40,
                              seed = 5,
                              lossFunction = "multinomial")

print("Training Classification GBT done")

[1] "Training Classification GBT done"


In [184]:
# Save the GBT in SQL. The Compute Context is set to Local in order to export the model. 
rxSetComputeContext(local)
saveRDS(btree_model_class, file = "btree_model_class.rds")
btree_model_class_raw <- readBin("btree_model_class.rds", "raw", n = file.size("btree_model_class.rds"))
btree_model_class_char <- as.character(btree_model_class_raw)
btree_model_class_sql <- RxSqlServerData(table = "Btree_Model_Class", connectionString = connection_string) 
rxDataStep(inData = data.frame(x = btree_model_class_char ), outFile = btree_model_class_sql, overwrite = TRUE)

print("Classification GBT model uploaded to SQL")

Rows Read: 811647, Total Rows Processed: 811647
Total Rows written: 100000, Total time: 2.374
Total Rows written: 200000, Total time: 4.638
Total Rows written: 300000, Total time: 6.856
Total Rows written: 400000, Total time: 9.105
Total Rows written: 500000, Total time: 11.307
Total Rows written: 600000, Total time: 13.541
Total Rows written: 700000, Total time: 15.884
Total Rows written: 800000, Total time: 18.211
Total Rows written: 811647, Total time: 18.523
, Total Chunk Time: 18.555 seconds 
[1] "Classification GBT model uploaded to SQL"


In [185]:
# Multi-class classification model evaluation metrics

evaluate_model_class <- function(observed, predicted, model) {
  confusion <- table(observed, predicted)
  num_classes <- nlevels(observed)
  tp <- rep(0, num_classes)
  fn <- rep(0, num_classes)
  fp <- rep(0, num_classes)
  tn <- rep(0, num_classes)
  accuracy <- rep(0, num_classes)
  precision <- rep(0, num_classes)
  recall <- rep(0, num_classes)
  for(i in 1:num_classes) {
    tp[i] <- sum(confusion[i, i])
    fn[i] <- sum(confusion[-i, i])
    fp[i] <- sum(confusion[i, -i])
    tn[i] <- sum(confusion[-i, -i])
    accuracy[i] <- (tp[i] + tn[i]) / (tp[i] + fn[i] + fp[i] + tn[i])
    precision[i] <- tp[i] / (tp[i] + fp[i])
    recall[i] <- tp[i] / (tp[i] + fn[i])
  }
  overall_accuracy <- sum(tp) / sum(confusion)
  average_accuracy <- sum(accuracy) / num_classes
  micro_precision <- sum(tp) / (sum(tp) + sum(fp))
  macro_precision <- sum(precision) / num_classes
  micro_recall <- sum(tp) / (sum(tp) + sum(fn))
  macro_recall <- sum(recall) / num_classes
  metrics <- c("Overall accuracy" = overall_accuracy,
               "Average accuracy" = average_accuracy,
               "Micro-averaged Precision" = micro_precision,
               "Macro-averaged Precision" = macro_precision,
               "Micro-averaged Recall" = micro_recall,
               "Macro-averaged Recall" = macro_recall)
  print(model)
  print(metrics)
  print(confusion)
  return(metrics)
}

In [186]:
# Classification Random Forest Scoring

## Make Predictions, then import them into R. The observed Conversion_Flag is kept through the argument extraVarsToWrite.
Prediction_Table_RF_Class <- RxSqlServerData(table = "Forest_Prediction_Class", stringsAsFactors = T, connectionString = connection_string)
rxPredict(forest_model_class, data = LoS_Test, outData = Prediction_Table_RF_Class, overwrite = T, type = "prob",
          extraVarsToWrite = c("lengthofstay"))

Prediction_RF_Class <- rxImport(inData = Prediction_Table_RF_Class, stringsAsFactors = T, outFile = NULL)

## Compute the performance metrics of the model.
Metrics_RF_Class <- evaluate_model_class(observed = factor(Prediction_RF_Class$lengthofstay, levels = c("1","2","3","4")),
                                         predicted = factor(Prediction_RF_Class$lengthofstay_Pred, levels = c("1","2","3","4")),
                                         model = "RF")

print("Scoring Classification RF done")

Rows Read: 21675, Total Rows Processed: 21675, Total Chunk Time: 0.766 seconds
Total Rows written: 21675, Total time: 0.594
 
Rows Read: 21675, Total Rows Processed: 21675, Total Chunk Time: 0.109 seconds 
[1] "RF"
        Overall accuracy         Average accuracy Micro-averaged Precision 
               0.8355709                0.9177855                0.8355709 
Macro-averaged Precision    Micro-averaged Recall    Macro-averaged Recall 
               0.8358130                0.8355709                0.8544421 
        predicted
observed    1    2    3    4
       1 5178  233    0    0
       2 1578 2103 1747    0
       3    0    6 5337    0
       4    0    0    0 5493
[1] "Scoring Classification RF done"


In [187]:
# Classification Gradient Boosted Trees Scoring 

## Make Predictions, then import them into R. The observed Conversion_Flag is kept through the argument extraVarsToWrite.
Prediction_Table_GBT_Class <- RxSqlServerData(table = "Boosted_Prediction_Class", stringsAsFactors = T, connectionString = connection_string)
rxPredict(btree_model_class,data = LoS_Test, outData = Prediction_Table_GBT_Class, overwrite = T, type="prob",
          extraVarsToWrite = c("lengthofstay"))

Prediction_GBT_Class <- rxImport(inData = Prediction_Table_GBT_Class, stringsAsFactors = T, outFile = NULL)

## Compute the performance metrics of the model.
Metrics_GBT_Class <- evaluate_model_class(observed = factor(Prediction_GBT_Class$lengthofstay, levels = c("1","2","3","4")),
                                          predicted = factor(Prediction_GBT_Class$lengthofstay_Pred, levels = c("1","2","3","4")),
                                          model = "GBT")

print("Scoring Classification GBT done")

Rows Read: 21675, Total Rows Processed: 21675, Total Chunk Time: 0.766 seconds
Total Rows written: 21675, Total time: 0.609
 
Rows Read: 21675, Total Rows Processed: 21675, Total Chunk Time: 0.078 seconds 
[1] "GBT"
        Overall accuracy         Average accuracy Micro-averaged Precision 
               0.8355709                0.9177855                0.8355709 
Macro-averaged Precision    Micro-averaged Recall    Macro-averaged Recall 
               0.8358511                0.8355709                0.8755483 
        predicted
observed    1    2    3    4
       1 5411    0    0    0
       2 1809 1864 1755    0
       3    0    0 5343    0
       4    0    0    0 5493
[1] "Scoring Classification GBT done"


## Step 3-B: Training and Evaluating the Models: Regression Approach

In this step we:

**1.** Split LoS into a Training LoS_Train, and a Testing set LoS_Test.  

**2.** Train regression Random Forest (RF) and Gradient Boosting Trees (GBT) on LoS_Train, and save them to SQL. 

**3.** Score RF and GBT on LoS_Test.

**Input:** Data set LoS

**Output:** Random forest and GBT models saved to SQL. 

In [188]:
# Point to the data set.
LoS <- RxSqlServerData(table = "LoS", connectionString = connection_string, stringsAsFactors = T)

# The target variable is converted to integer for regression.
rxExecuteSQLDDL(outOdbcDS, sSQLString = paste("ALTER TABLE LoS ALTER COLUMN lengthofstay int;", sep=""))

# Get the column info.
column_info <- rxCreateColInfo(LoS)

print("Column information received")

[1] TRUE

[1] "Column information received"


In [189]:
# Randomly split the data into a training set and a testing set, with a splitting % p.
# p % goes to the training set, and the rest goes to the testing set. Default is 70%. 

p <- "70" 

## Open a connection with SQL Server to be able to write queries with the rxExecuteSQLDDL function.
outOdbcDS <- RxOdbcData(table = "NewData", connectionString = connection_string, useFastRead=TRUE)
rxOpen(outOdbcDS, "w")

## Create the Train_Id table containing Lead_Id of training set. 
rxExecuteSQLDDL(outOdbcDS, sSQLString = paste("DROP TABLE if exists Train_Id;", sep=""))

rxExecuteSQLDDL(outOdbcDS, sSQLString = sprintf(
  "SELECT eid
   INTO Train_Id
   FROM LoS
   WHERE ABS(CAST(BINARY_CHECKSUM(eid, NEWID()) as int)) %s < %s ;"
  ,"% 100", p ))

## Point to the training set. It will be created on the fly when training models. 
LoS_Train <- RxSqlServerData(  
  sqlQuery = "SELECT *   
              FROM LoS 
              WHERE eid IN (SELECT eid from Train_Id)",
  connectionString = connection_string, colInfo = column_info)

## Point to the testing set. It will be created on the fly when testing models. 
LoS_Test <- RxSqlServerData(  
  sqlQuery = "SELECT *   
              FROM LoS 
              WHERE eid NOT IN (SELECT eid from Train_Id)",
  connectionString = connection_string, colInfo = column_info)

print("Splitting completed")

[1] TRUE

[1] TRUE

[1] TRUE

[1] "Splitting completed"


In [190]:
# Compute Context is set to SQL for model training.
rxSetComputeContext(sql)

In [191]:
# Train a Random Forest.
forest_model_reg <- rxDForest(formula = formula,
                              data = LoS_Train,
                              nTree = 40,
                              minSplit = 10,
                              minBucket = 5,
                              cp = 0.00005,
                              seed = 5)

print("Training Regression RF done")

[1] "Training Regression RF done"


In [192]:
# Save the Random Forest in SQL. The compute context is set to Local in order to export the model. 
rxSetComputeContext(local)
saveRDS(forest_model_reg, file = "forest_model_reg.rds")
forest_model_reg_raw <- readBin("forest_model_reg.rds", "raw", n = file.size("forest_model_reg.rds"))
forest_model_reg_char <- as.character(forest_model_reg_raw)
forest_model_reg_sql <- RxSqlServerData(table = "Forest_Model_Reg", connectionString = connection_string) 
rxDataStep(inData = data.frame(x = forest_model_reg_char ), outFile = forest_model_reg_sql, overwrite = TRUE)

# Set back the compute context to SQL.
rxSetComputeContext(sql)

print("RF Regression model uploaded to SQL")

Rows Read: 1075729, Total Rows Processed: 1075729
Total Rows written: 100000, Total time: 2.14
Total Rows written: 200000, Total time: 4.342
Total Rows written: 300000, Total time: 6.529
Total Rows written: 400000, Total time: 8.731
Total Rows written: 500000, Total time: 11.027
Total Rows written: 600000, Total time: 13.292
Total Rows written: 700000, Total time: 15.51
Total Rows written: 800000, Total time: 17.712
Total Rows written: 900000, Total time: 19.898
Total Rows written: 1000000, Total time: 22.101
Total Rows written: 1075729, Total time: 23.788
, Total Chunk Time: 23.850 seconds 
[1] "RF Regression model uploaded to SQL"


In [193]:
# Train the GBT.
btree_model_reg <- rxBTrees(formula = formula,
                            data = LoS_Train,
                            learningRate = 0.05,
                            minSplit = 10,
                            minBucket = 5,
                            cp = 0.0005,
                            nTree = 40,
                            seed = 5,
                            lossFunction = "gaussian")

print("Training Regression GBT done")

[1] "Training Regression GBT done"


In [194]:
# Save the GBT in SQL. The Compute Context is set to Local in order to export the model. 
rxSetComputeContext(local)
saveRDS(btree_model_reg, file = "btree_model_reg.rds")
btree_model_reg_raw <- readBin("btree_model_reg.rds", "raw", n = file.size("btree_model_reg.rds"))
btree_model_reg_char <- as.character(btree_model_reg_raw)
btree_model_reg_sql <- RxSqlServerData(table = "Btree_Model_Reg", connectionString = connection_string) 
rxDataStep(inData = data.frame(x = btree_model_reg_char ), outFile = btree_model_reg_sql, overwrite = TRUE)

print("GBT Regression model uploaded to SQL")

Rows Read: 800762, Total Rows Processed: 800762
Total Rows written: 100000, Total time: 2.343
Total Rows written: 200000, Total time: 5.685
Total Rows written: 300000, Total time: 7.888
Total Rows written: 400000, Total time: 10.231
Total Rows written: 500000, Total time: 12.417
Total Rows written: 600000, Total time: 14.651
Total Rows written: 700000, Total time: 16.947
Total Rows written: 800000, Total time: 19.196
Total Rows written: 800762, Total time: 19.259
, Total Chunk Time: 19.275 seconds 
[1] "GBT Regression model uploaded to SQL"


In [195]:
# Write a function that computes regression performance metrics. 
evaluate_model_reg <- function(observed, predicted, model) {
  
  print(model)
  ## Regression Metrics
  mean_observed <- mean(observed)
  se <- (observed - predicted)^2
  ae <- abs(observed - predicted)
  sem <- (observed - mean_observed)^2
  aem <- abs(observed - mean_observed)
  mae <- mean(ae)
  rmse <- sqrt(mean(se))
  rae <- sum(ae) / sum(aem)
  rse <- sum(se) / sum(sem)
  rsq <- 1 - rse
  
  ## Classification Metrics when the prediction is rounded to the nearest integer. 
  predicted <- factor(round(predicted), levels = c("1","2","3","4"))
  observed <- factor(observed, levels = c("1","2","3","4"))
  
  confusion <- table(observed, predicted)
  num_classes <- nlevels(observed)
  tp <- rep(0, num_classes)
  fn <- rep(0, num_classes)
  fp <- rep(0, num_classes)
  tn <- rep(0, num_classes)
  accuracy <- rep(0, num_classes)
  precision <- rep(0, num_classes)
  recall <- rep(0, num_classes)
  for(i in 1:num_classes) {
    tp[i] <- sum(confusion[i, i])
    fn[i] <- sum(confusion[-i, i])
    fp[i] <- sum(confusion[i, -i])
    tn[i] <- sum(confusion[-i, -i])
    accuracy[i] <- (tp[i] + tn[i]) / (tp[i] + fn[i] + fp[i] + tn[i])
    precision[i] <- tp[i] / (tp[i] + fp[i])
    recall[i] <- tp[i] / (tp[i] + fn[i])
  }
  overall_accuracy <- sum(tp) / sum(confusion)
  average_accuracy <- sum(accuracy) / num_classes
  micro_precision <- sum(tp) / (sum(tp) + sum(fp))
  macro_precision <- sum(precision) / num_classes
  micro_recall <- sum(tp) / (sum(tp) + sum(fn))
  macro_recall <- sum(recall) / num_classes
  
  ## Writing all the performance metrics. 
  metrics <- c("Mean Absolute Error" = mae,
               "Root Mean Squared Error" = rmse,
               "Relative Absolute Error" = rae,
               "Relative Squared Error" = rse,
               "Coefficient of Determination" = rsq,
               "Overall accuracy (Rounded Prediction)" = overall_accuracy,
               "Average accuracy (Rounded Prediction)" = average_accuracy,
               "Micro-averaged Precision (Rounded Prediction)" = micro_precision,
               "Macro-averaged Precision (Rounded Prediction)" = macro_precision,
               "Micro-averaged Recall (Rounded Prediction)" = micro_recall,
               "Macro-averaged Recall (Rounded Prediction)" = macro_recall)
  print(metrics)
  print("Confusion Matrix when the prediction is rounded")
  print(confusion)
  return(metrics)
}

In [196]:
# Regression Random Forest Scoring 

## Make Predictions, then import them into R. The observed Conversion_Flag is kept through the argument extraVarsToWrite.
Prediction_Table_RF_Reg <- RxSqlServerData(table = "Forest_Prediction_Reg", stringsAsFactors = T, connectionString = connection_string)
rxPredict(forest_model_reg, data = LoS_Test, outData = Prediction_Table_RF_Reg, overwrite = T, type = "response",
          extraVarsToWrite = c("lengthofstay"))

Prediction_RF_Reg<- rxImport(inData = Prediction_Table_RF_Reg, stringsAsFactors = T, outFile = NULL)

## Compute the performance metrics of the model.
Metrics_RF_Reg <- evaluate_model_reg(observed = Prediction_RF_Reg$lengthofstay,
                                    predicted = Prediction_RF_Reg$lengthofstay_Pred,
                                    model = "RF")
print("Scoring Regression RF done")

Rows Read: 21799, Total Rows Processed: 21799, Total Chunk Time: 0.797 seconds
Total Rows written: 21799, Total time: 0.406
 
Rows Read: 21799, Total Rows Processed: 21799, Total Chunk Time: 0.016 seconds 
[1] "RF"
                          Mean Absolute Error 
                                   0.22376645 
                      Root Mean Squared Error 
                                   0.30176265 
                      Relative Absolute Error 
                                   0.22428126 
                       Relative Squared Error 
                                   0.07311492 
                 Coefficient of Determination 
                                   0.92688508 
        Overall accuracy (Rounded Prediction) 
                                   0.83398321 
        Average accuracy (Rounded Prediction) 
                                   0.91699161 
Micro-averaged Precision (Rounded Prediction) 
                                   0.83398321 
Macro-averaged Precision (Rounded

In [197]:
# Regression Gradient Boosted Trees Scoring 

## Make Predictions, then import them into R. The observed Conversion_Flag is kept through the argument extraVarsToWrite.
Prediction_Table_GBT_Reg <- RxSqlServerData(table = "Boosted_Prediction_Reg", stringsAsFactors = T, connectionString = connection_string)
rxPredict(btree_model_reg,data = LoS_Test, outData = Prediction_Table_GBT_Reg, overwrite = T, type="response",
          extraVarsToWrite = c("lengthofstay"))

Prediction_GBT_Reg <- rxImport(inData = Prediction_Table_GBT_Reg, stringsAsFactors = T, outFile = NULL)

## Compute the performance metrics of the model.
Metrics_GBT_Reg <- evaluate_model_reg(observed = Prediction_GBT_Reg$lengthofstay,
                                      predicted = Prediction_GBT_Reg$lengthofstay_Pred,
                                      model = "GBT")

print("Scoring regression GBT done")

Rows Read: 21799, Total Rows Processed: 21799, Total Chunk Time: 0.765 seconds
Total Rows written: 21799, Total time: 0.406
 
Rows Read: 21799, Total Rows Processed: 21799, Total Chunk Time: 0.015 seconds 
[1] "GBT"
                          Mean Absolute Error 
                                    0.6274274 
                      Root Mean Squared Error 
                                    0.7384955 
                      Relative Absolute Error 
                                    0.6288709 
                       Relative Squared Error 
                                    0.4378957 
                 Coefficient of Determination 
                                    0.5621043 
        Overall accuracy (Rounded Prediction) 
                                    0.4212579 
        Average accuracy (Rounded Prediction) 
                                    0.7106289 
Micro-averaged Precision (Rounded Prediction) 
                                    0.4212579 
Macro-averaged Precision (Rounde